# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [2]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [3]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [4]:
# We group data by CustomerID and ProductID and sum Quantity, although only numeric columns will be summed 
# (if we include SalesID, these values will be summed too)
data_group = data[["CustomerID", "ProductID", "Quantity"]].groupby(["CustomerID", "ProductID"]).sum()
data_group.head()

Quantity
CustomerID ProductID          
33         23                1
           51                1
           53                1
           67                1
           70                1

In [5]:
# We are creating a multiindex df, that's how we access to each row:
#data_group.loc[(33,443)]

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [6]:
# Create a df with the summed values of Quantity, and set as index the ProductID and as columns, the CustomerID
customer_mtx = pd.pivot_table(data, values = "Quantity", index = "ProductID", columns = "CustomerID")

# Fill NaN with 0
customer_mtx = customer_mtx.fillna(0)
customer_mtx.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductID,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0,0.0,25.0
2,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,...,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [7]:
# We calculate the distance from one customer to another using the euclidean distance. If we had wanted to 
# calculate the distance between the products (e.g. to categorize them), we would have used "customer_mtx"
distances = pdist(customer_mtx.T, 'euclidean') # shape = (499500,) nº eltos en uno de los triángulos de la mtx

# reshape the array distances into a square matrix
distances = squareform(distances)

# Similarity = 1/(1+distance)
sim_mtx = pd.DataFrame(1/(1 + distances), 
                       index=customer_mtx.columns, 
                       columns=customer_mtx.columns)

sim_mtx.head() # shape (1000, 1000)

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.085297,0.093953,0.091747,0.087410,0.089695,0.085297,0.088913,0.088152,0.089695,...,0.004809,0.005108,0.004996,0.005421,0.004920,0.005023,0.004880,0.005026,0.004549,0.004883
200,0.085297,1.000000,0.085638,0.085297,0.080070,0.083020,0.084959,0.083651,0.085638,0.087047,...,0.004825,0.005121,0.005014,0.005448,0.004925,0.005032,0.004909,0.005042,0.004553,0.004879
264,0.093953,0.085638,1.000000,0.088152,0.089301,0.087047,0.085638,0.086333,0.087047,0.087047,...,0.004822,0.005115,0.004996,0.005441,0.004932,0.005055,0.004894,0.005042,0.004566,0.004883
356,0.091747,0.085297,0.088152,1.000000,0.085983,0.086688,0.085983,0.091325,0.085983,0.087410,...,0.004814,0.005111,0.004999,0.005437,0.004920,0.005036,0.004871,0.005042,0.004563,0.004886
412,0.087410,0.080070,0.089301,0.085983,1.000000,0.085638,0.085638,0.089301,0.084959,0.087779,...,0.004808,0.005131,0.004996,0.005441,0.004925,0.005042,0.004876,0.005039,0.004568,0.004903


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [8]:
# Chose a customer
customer = 639

# Take the column in the sim_mtx for this customer, and sort the values in descending order. The first value
# will be always the customer himself.
sim_customer = sim_mtx[customer].sort_values(ascending=False)[1:]

# Top 5 most similar customers
top5 = sim_customer[0:5]
top5

CustomerID
1920    0.094414
3253    0.093498
3317    0.093498
649     0.093498
3472    0.093051
Name: 639, dtype: float64

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [9]:
# Get the indices for the top5 most similar customers
top5_idx = sim_customer[0:5].index

# Select the rows related to the top5 similar customers from df "data_group"
data_top5_group = data_group.loc[(top5_idx)] # To access data by row
data_top5_group.head()

Quantity
CustomerID ProductID          
649        3                 1
           6                 1
           8                 2
           27                1
           37                1

## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [10]:
ranked_products = data_top5_group.groupby("ProductID").sum().sort_values("Quantity", ascending = False)
ranked_products = ranked_products.reset_index()

## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [11]:
# Create a df from the customer's column in customer_mtx
one_customer_mtx = pd.DataFrame(customer_mtx[customer])
one_customer_mtx = one_customer_mtx.reset_index()

# Merge ranked products and customer df on the "ProductID" column, as the ranked products df is already sorted
# by quantity, merged_mtx will be sorted by quantity too
merged_mtx = ranked_products.merge(one_customer_mtx, on= "ProductID")

# Products not purchased by the customer
not_puchased_mtx = merged_mtx.loc[merged_mtx[customer]==0]
not_puchased_mtx

# top5 products
top5_prod = not_puchased_mtx.sort_values("Quantity", ascending = False).reset_index(drop = True)[0:5]
top5_prod

,ProductID,Quantity,639
0,215,3,0.0
1,279,3,0.0
2,195,3,0.0
3,83,3,0.0
4,451,3,0.0


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [12]:
def recommended_products(customer, sim_mtx):
    # Top 5 most similar customers: get the indices
    top5_idx = sim_mtx[customer].sort_values(ascending=False)[1:6].index

    # Select the rows related to the top5 similar customers from df "data_group"
    data_top5_group = data_group.loc[(top5_idx)]
    
    # Rank products purchased by the top5 similar customers
    ranked_products = data_top5_group.groupby("ProductID").sum().sort_values("Quantity", ascending = False)
    ranked_products = ranked_products.reset_index()
    
    # Create a df from the customer's column in customer_mtx
    one_customer_mtx = pd.DataFrame(customer_mtx[customer])
    one_customer_mtx = one_customer_mtx.reset_index()

    # Merge ranked products and customer df on the "ProductID" column
    merged_mtx = ranked_products.merge(one_customer_mtx, on= "ProductID")

    # Products not purchased by the customer
    not_puchased_mtx = merged_mtx.loc[merged_mtx[customer]==0]

    # top5 products
    top5_prod = not_puchased_mtx.sort_values("Quantity", ascending = False).reset_index(drop = True)[0:5]

    return {
        "Customer": customer,
        "Top1": top5_prod["ProductID"][0], "Top2": top5_prod["ProductID"][1], 
        "Top3": top5_prod["ProductID"][2], "Top4": top5_prod["ProductID"][3], 
        "Top5": top5_prod["ProductID"][4]
    }

In [13]:
# List of unique customers
unique_customers = list(set(data["CustomerID"]))

In [14]:
recom_prod_list = [recommended_products(e, sim_mtx) for e in unique_customers]

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [15]:
df_top5 = pd.DataFrame(recom_prod_list).set_index("Customer")
df_top5.head()

,Top1,Top2,Top3,Top4,Top5
Customer,,,,,
83973,313,12,93,206,60
59399,37,412,252,2,276
92168,48,273,130,414,345
49159,302,81,12,192,316
18441,208,449,97,227,127


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [16]:
# Recalculating the distance with correlation
distances2 = pdist(customer_mtx.T, 'correlation') # shape = (499500,)

# Similarity = 1/(1+distance)
sim_mtx2 = pd.DataFrame(1/(1 + squareform(distances2)), 
                       index=customer_mtx.columns, 
                       columns=customer_mtx.columns)
sim_mtx2.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.493805,0.529875,0.515035,0.500570,0.504367,0.487132,0.492406,0.494838,0.500815,...,0.491005,0.486159,0.498339,0.480313,0.497157,0.480750,0.492063,0.485173,0.482914,0.496579
200,0.493805,1.000000,0.500545,0.496849,0.472695,0.483532,0.504988,0.481550,0.500545,0.506509,...,0.508587,0.496849,0.517060,0.506602,0.497813,0.487096,0.528445,0.499846,0.483420,0.484852
264,0.529875,0.500545,1.000000,0.498278,0.517080,0.493067,0.494101,0.481681,0.493067,0.489652,...,0.511597,0.493648,0.496434,0.502956,0.513787,0.526176,0.513045,0.506386,0.510648,0.494672
356,0.515035,0.496849,0.498278,1.000000,0.494727,0.489104,0.494727,0.510742,0.484642,0.490190,...,0.498600,0.489643,0.501690,0.498569,0.495876,0.497664,0.478039,0.507100,0.507023,0.499809
412,0.500570,0.472695,0.517080,0.494727,1.000000,0.494101,0.503354,0.510747,0.489748,0.504478,...,0.485869,0.513118,0.492877,0.499410,0.500310,0.502343,0.482847,0.497874,0.510144,0.522382


In [17]:
recom_prod_list2 = [recommended_products(e, sim_mtx2) for e in unique_customers]
df_top5_v2 = pd.DataFrame(recom_prod_list2).set_index("Customer")
df_top5_v2.head()

,Top1,Top2,Top3,Top4,Top5
Customer,,,,,
83973,208,445,17,89,390
59399,200,12,131,63,37
92168,24,252,291,307,63
49159,358,29,359,125,7
18441,334,433,396,175,358


**Conclusión**: Al calcular la correlación, estamos teniendo en cuenta cuántas unidades de media se compran de cada producto. Es decir, si hay un producto que se vende poco, los clientes A y B que lo compren, estarán más correlacionados (tendrán más porcentaje de similitud), y si por el contrario, hay un producto que se vende mucho, la correlación no será tan importante.